# Wir implementieren einen Hund-Katze Klassifikator mit neuronalen Netzen

In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense
from tensorflow.keras import utils
import numpy as np
import argparse
import cv2
import os

## Folgende Funktion nimmt ein Bild entgegen, skaliert das Bild auf 32x32 RGB-Pixel, und liefert einen 3072-dim Merkmalevektor zurueck

In [2]:
def image_to_feature_vector(image, size=(32, 32)):
    return cv2.resize(image, size).flatten()

## Laden Sie die Daten aus dem gitlab:


Schreiben Sie eine Funktion, die die Bilder einliest, mit Hilfe der Funktion image_to_feature_vector vorverarbeiten. Anschliessend sollen die Vektoren in eine data-Matrix geschrieben werden, 
und ein label-Vektor ist zu erzeugen. Die labels muessen entweder 0 oder 1 sein

In [3]:

def gen_data(path):
    data = []
    labels = []

    imagePaths = [os.path.join(path, file) for file in os.listdir(path) if file.endswith('.jpg')]

    for (i, imagePath) in enumerate(imagePaths):
        image = cv2.imread(imagePath)
        label = os.path.basename(imagePath).split(".")[0]
        features = image_to_feature_vector(image)
        data.append(features)
        labels.append(label)
 
    return data, labels

In [4]:
data, labels = gen_data('...\\train')

le = LabelEncoder()
labels = le.fit_transform(labels)

print(labels)
print(labels.shape)

[0 0 0 ... 1 1 1]
(25000,)


## Splitten Sie die Daten in ein train/test-set.

Macht es Sinn die Daten zu normieren?

In [5]:
data = np.array(data) / 255.0
print(data.shape)

labels = utils.to_categorical(labels, 2)
print(labels.shape)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, random_state=42)

(25000, 3072)
(25000, 2)


# Erstellen Sie ein Neuronales Netz mit Keras

In [6]:
model = Sequential()
model.add(Dense(768, input_dim=3072, activation="relu"))
model.add(Dense(384, activation="relu"))
model.add(Dense(2)) 
model.add(Activation('softmax'))

# ... und starten Sie das Training

In [8]:
sgd = SGD(learning_rate=0.01)
model.compile(loss="binary_crossentropy", optimizer=sgd, metrics=["accuracy"])
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

Epoch 1/50
586/586 [==============================] - 5s 8ms/step - loss: 0.6758 - accuracy: 0.5756
Epoch 2/50
586/586 [==============================] - 5s 9ms/step - loss: 0.6556 - accuracy: 0.6116
Epoch 3/50
586/586 [==============================] - 5s 8ms/step - loss: 0.6436 - accuracy: 0.6268
Epoch 4/50
586/586 [==============================] - 5s 8ms/step - loss: 0.6335 - accuracy: 0.6421
Epoch 5/50
586/586 [==============================] - 4s 8ms/step - loss: 0.6285 - accuracy: 0.6434
Epoch 6/50
586/586 [==============================] - 5s 9ms/step - loss: 0.6199 - accuracy: 0.6542
Epoch 7/50
586/586 [==============================] - 4s 7ms/step - loss: 0.6124 - accuracy: 0.6617
Epoch 8/50
586/586 [==============================] - 5s 8ms/step - loss: 0.6061 - accuracy: 0.6693
Epoch 9/50
586/586 [==============================] - 6s 9ms/step - loss: 0.5997 - accuracy: 0.6722
Epoch 10/50
586/586 [==============================] - 4s 7ms/step - loss: 0.5942 - accuracy: 0.6791

# Evaluieren Sie das Netz auf den held-back Test-Daten


Berechnen Sie die Akkuratheit und die Verwechselungsmatrix

In [10]:
loss, accuracy = model.evaluate(X_test, y_test, batch_size=128, verbose=0)
print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

model.save_weights('model_weights.h5')
model.save('model_keras.h5')

Loss: 0.6793, Accuracy: 0.6707


# Klassifizieren Sie zusätzlich die Daten aus test100 und schauen Sie sich die Ergebnisse an

In [11]:
classes = ["cat", "dog"]
directory = "...\\test100"

for file in os.listdir(directory):
    imagePath = os.path.join(directory, file)
    
    print(f"classifying {file}")
    image = cv2.imread(imagePath)
    features = image_to_feature_vector(image) / 255.0
    features = np.array([features])
    
    probs = model.predict(features)[0]
    prediction = probs.argmax(axis=0)
 
    label = f"{classes[prediction]}: {probs[prediction] * 100:.2f}%"
    print(label)

classifying cat.0.jpg
1/1 [==============================] - 0s 79ms/step
dog: 86.99%
classifying cat.1.jpg
1/1 [==============================] - 0s 20ms/step
cat: 99.52%
classifying cat.10.jpg
1/1 [==============================] - 0s 20ms/step
cat: 82.30%
classifying cat.11.jpg
1/1 [==============================] - 0s 20ms/step
cat: 89.60%
classifying cat.12.jpg
1/1 [==============================] - 0s 20ms/step
cat: 84.75%
classifying cat.13.jpg
1/1 [==============================] - 0s 21ms/step
cat: 87.51%
classifying cat.14.jpg
1/1 [==============================] - 0s 20ms/step
cat: 96.66%
classifying cat.15.jpg
1/1 [==============================] - 0s 22ms/step
cat: 94.88%
classifying cat.16.jpg
1/1 [==============================] - 0s 20ms/step
cat: 79.46%
classifying cat.17.jpg
1/1 [==============================] - 0s 23ms/step
cat: 51.50%
classifying cat.18.jpg
1/1 [==============================] - 0s 21ms/step
dog: 62.52%
classifying cat.19.jpg
1/1 [=================